## RedisDB

In [1]:
import redis
import json
import pandas as pd
import RedisDB
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

In [2]:
rdb = RedisDB.RedisDB()

In [3]:
pool = redis.ConnectionPool(host='localhost',  port=6379, decode_responses=True)
r = redis.Redis(connection_pool=pool)

In [4]:
Get = RedisDB.RedisGet()

In [5]:
Get.cal_topic_index_rtn()

AttributeError: 'RedisGet' object has no attribute 'my_dic'

In [ ]:
stk_list = Get.get_stk_list()

In [ ]:
len(stk_list)

In [ ]:
Get.get_stk(['sz000001'])

In [ ]:
pd.DataFrame(Get.mDataDict).transpose().sort_index()

In [ ]:
df = Get.mDataFrame.sort_index()

In [ ]:
df

In [ ]:
# 计算vol
df_1['vol'] = df_1['volume'].diff(1)
df_1['vol'][0] = df_1['volume'][0]

In [ ]:
# 计算vwap
def vwap(df):
    v = df['vol'].values
    tp = (df['low'] + df['now'] + df['high']).div(3).values
    return df.assign(vwap=(tp*v).cumsum()/v.cumsum())
df_1 = vwap(df_1)

In [ ]:
df_1[['high','low','now','vwap']].plot(figsize=[20,3],color = color_lst)

In [ ]:
import datetime

In [ ]:
r.config_set('dir','D:\\Pzh\\RedisData')
date_str = str(datetime.date.today())
r.config_set('dbfilename',date_str+'.rdb')

In [ ]:
r.save()

## Quotation

In [ ]:
%pip install WindPy

In [ ]:
import os,time
from datetime import datetime
import numpy as np
import pandas as pd
import easyquotation
#import deal_topic as dt
from loguru import logger

In [ ]:
quotation = easyquotation.use('sina') # 新浪 ['sina'] 腾讯 ['tencent', 'qq']
last_snap = quotation.market_snapshot(prefix=True)

def cal_topic_index_rtn(chinese_topics,last_snapshort):
    # 计算结果和wind的一样
    top_index_rtn = dict()
    for ct in chinese_topics:
        ct_tickers = chinese_topics[ct]
        ct_rtns = list()
        for ctt in ct_tickers:
            if ctt.startswith('sh8'):continue  # 北交所的票忽略
            ctt_rtn = last_snapshort[ctt]['now']/last_snapshort[ctt]['close']-1
            ct_rtns.append(ctt_rtn)
        # print(ct,np.mean(ct_rtns))
        top_index_rtn[ct] = np.mean(ct_rtns)
    return top_index_rtn

In [ ]:
import os,time
import numpy as np
import pandas as pd


def deal_for_easyquotation():
    # 处理成easyquotation能读取的格式

    chinese_map = pd.read_csv('./txt/top_chinese_map.csv',encoding='gbk')
    chinese_map = dict(zip(list(chinese_map.iloc[:,0]),list(chinese_map.iloc[:,1])))
    topics = os.listdir(r"./topic")
    chinese_topics = dict()
    for topi in topics:
        topi_data = pd.read_csv(fr"./topic/{topi}")
        tickers = ['sz'+t[:6] if t[-1]=='Z' else 'sh'+t[:6] for t in topi_data['1'].values]
        chinese_topics[chinese_map[topi.split('.')[0]]] = tickers
    return chinese_topics


In [ ]:
chinese_topics = deal_for_easyquotation()

In [ ]:
cal_topic_index_rtn(chinese_topics,last_snap)

In [ ]:
topics = os.listdir(r"./topic")
for topi in topics:
    topi_data = pd.read_csv(fr"./topic/{topi}")
        

In [ ]:
topi_data